In [ ]:
import os
import pandas as pd  
print(os.getcwd())
## Now read in the generate text function from the primary script
from primary.generate_text import generate_text

In [ ]:

## Prepare models
model = "./model/Meta-Llama-3-8B-Instruct-Q3_K_S.gguf"  


In [ ]:

## Prepare School data for analysis
# - Load the data from the Excel file
data_directory = "./data/"
# data_directory = "C:/Users/pkress/projects/REL-DCPS-workshop/REL-DCPS-NLP-public/data/"
input_path = data_directory + "Intervention Sample_lbNotes.csv"
goal_data = pd.read_csv(input_path)
goal_data.info()

In [ ]:

# - Preliminary cleaning
keep_cols = ["intervention_id", "subject_area", "grade_level", "school_id"
             , "group_intervention_name", "strategies", "strategy_ids", "tier"
             , "in_group", "plan_goal"]
cln_data = goal_data.dropna(
    subset=["plan_goal"]
    ).loc[
    :, keep_cols
    ]


In [ ]:

# - Some initial exploration
print(cln_data.groupby(['grade_level', "subject_area"]).intervention_id.count())


In [ ]:


print(cln_data.groupby(['grade_level']).intervention_id.count())

In [ ]:
print(cln_data.loc[cln_data.grade_level=="6"].groupby(['subject_area']).intervention_id.count())


In [ ]:
print(cln_data.loc[cln_data.grade_level=="11"].groupby(['subject_area']).intervention_id.count())

In [ ]:

print(cln_data.loc[cln_data.grade_level=="C7"].groupby(['subject_area']).intervention_id.count())

In [ ]:

## Confirm intervention_id is unique
cln_data.intervention_id.nunique() == cln_data.shape[0]


In [ ]:
## Get some goals!
math_prompts = cln_data.loc[
    (cln_data.grade_level=="6") & (cln_data.subject_area=="Mathematics")
    ].sample(
    3
    ).loc[
    :, ["intervention_id", "plan_goal"]
    ] 
behave_prompts = cln_data.loc[
    (cln_data.grade_level=="6") & (cln_data.subject_area=="Behavior")
    ].sample(
    3
    ).loc[
    :, ["intervention_id", "plan_goal"]
    ]
print(behave_prompts)
    

In [ ]:

## Simple "Smart" feedback
system_prompt = """
You give feedback on the quality of the goals teachers set for their students. 
Treat all submitted text as a goal. 
Goals should be specific, measurable, achievable, and have a deadline.
Give feedback on the extent to which goals are specific, measurable, achievable, or have a deadline.
Include a numeric score for each category from 1 (needs improvement) to 5 (excellent). 
"""  
math_simple = generate_text(math_prompts.plan_goal.tolist(),
                 system_prompt = system_prompt, 
                 model_path = model,
                 temperature = 0.0, 
                 n_ctx = 1024, max_tokens = 1024)  



[print("________\n", math_simple['prompts'][i], "\n", math_simple['responses'][i]) \
    for i in range(len(math_simple['responses']))]
print(math_simple)

In [ ]:

## Simple "Smart" feedback
system_prompt = """
You are an expert on goal quality. 
Give feedback on the quality of the goals teachers set for their students. 
Treat all submitted text as a goal. 
First, give feedback on the specificity based on how specific the goal is. 
Second, give feedback on the achievability, based on whether you could expect a 6th grader to behave. 
Include a numeric score for each category from 1 (needs improvement) to 5 (excellent). 
"""  
math_simple = generate_text(math_prompts.plan_goal.tolist(),
                 system_prompt = system_prompt, 
                 model_path = model,
                 temperature = 0.0, 
                 n_ctx = 1024, max_tokens = 1024)  



[print("________\n", math_simple['prompts'][i], "\n", math_simple['responses'][i]) \
    for i in range(len(math_simple['responses']))]
print(math_simple)